# Hourly ET₀ Calculator – FAO Penman-Monteith Method
This notebook calculates hourly reference evapotranspiration (ET₀) using weather station data.

In [ ]:
# 📌 Install required library
!pip install pysolar

In [ ]:
# 📚 Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import exp
from datetime import datetime
from pysolar.solar import get_altitude

In [ ]:
# 📥 Upload your file
from google.colab import files
uploaded = files.upload()

In [ ]:
# 📊 Read data
df = pd.read_csv(next(iter(uploaded)))
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Hour'])

In [ ]:
# Constants
Gsc = 0.0820
albedo = 0.23
lat = 36.5657  # Utsunomiya latitude
z = df['Elevation'].iloc[0]

In [ ]:
# FAO Penman-Monteith function
def calculate_eto(row):
    T = row['T_air']
    RH = row['RH']
    u2 = row['Wind_speed_2m']
    Rs = row['Solar_radiation']
    time = row['Datetime']
    es = 0.6108 * exp((17.27 * T) / (T + 237.3))
    ea = es * (RH / 100)
    delta = (4098 * es) / ((T + 237.3)**2)
    P = 101.3 * ((293 - 0.0065 * z)/293)**5.26
    gamma = 0.665e-3 * P
    Rns = (1 - albedo) * Rs
    Rnl = 0
    Rn = Rns - Rnl
    G = 0
    numerator = (0.408 * delta * (Rn - G)) + (gamma * (900 / (T + 273)) * u2 * (es - ea))
    denominator = delta + gamma * (1 + 0.34 * u2)
    eto = numerator / denominator
    return eto

In [ ]:
# Apply calculation
df['ET0'] = df.apply(calculate_eto, axis=1)

In [ ]:
# 📈 Plot
plt.figure(figsize=(10,5))
plt.plot(df['Datetime'], df['ET0'], marker='o')
plt.title('Hourly Reference Evapotranspiration (ET₀)')
plt.xlabel('Time')
plt.ylabel('ET₀ (mm/hr)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 💾 Save to CSV
output_filename = "eto_output.csv"
df.to_csv(output_filename, index=False)
files.download(output_filename)